# WST Spike Finder

Iterate through folders, looking for possible spikes in all supported GPS file types.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

This is very crude and simply looks for Doppler speeds in excess of 30 m/s (i.e. 108 km/h or 58.32 knots).

Such speeds are currently unrealistic for a windsurfer.

In [1]:
import os
import sys

import numpy as np

from shapely.geometry import Polygon, Point

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [2]:
courseCoordinates = [
    (42.9692693, 3.0479496),
    (42.9697398, 3.0449530),
    (42.9653296, 3.0436693),
    (42.9648592, 3.0466659)
]

course = Polygon(courseCoordinates)

# 30 knots
speedThreshold = 30 * 1852 / 3600

# 1.5 knots
saccThreshold = 1.5 * 1852 / 3600

def findSpikes():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, '..', 'wst-results', 'events', '2024', 'gpslogs')

    errors = {}
    spikes = {}
    
    totFiles = 0
    totPoints = 0
    totDist = 0

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()

            # 'MOR854PIE' in file and 
            if ext and ext in ['.oao']:
                filePath = os.path.join(root, file)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        maxSog = 0
                        sAcc = 0
                        hAcc = 0
                        hdop = 0
                        sats = 0
                        
                        points = (np.argwhere(track.data['sog'] > speedThreshold))
                        for i in points:
                            lat = track.data['lat'][i[0]]
                            lon = track.data['lon'][i[0]]
                            point = Point(lat, lon)

                            # sAcc > 1 m/s is approximately 2 knots
                            if course.contains(point) and track.data['ehve'][i[0]] > saccThreshold:
                                sog =  track.data['sog'][i[0]]
                                if sog > maxSog:
                                    maxSog = sog
                                    sAcc = track.data['ehve'][i[0]]
                                    hAcc = track.data['ehpe'][i[0]]
                                    hdop = track.data['hdop'][i[0]]
                                    sats = track.data['sat'][i[0]]
                    
                        if maxSog > 0:
                            msg = '{:.02f},{:.02f},{:.02f},{:.02f},{:.02f},{:.02f},{}'.format(
                                maxSog, maxSog * 3600 / 1852, maxSog * 3600 / 1000, sAcc, hAcc, hdop, sats)
                            #spikes[filePath.replace(projdir + '/', '')] = msg
                            spikes[os.path.basename(filePath)] = msg
                            print('S', end='')
                        else:
                            print('.', end='')
                        
                        totPoints += len(track.data['sog'])
                        if ext in ['.oao']:
                            totDist += track.data['sog'].sum() / 5
                        else:
                            totDist += track.data['sog'].sum()

                    totFiles += 1

                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    print(os.linesep * 2 + 'Summary: files = {:,}, points = {:,}, distance = {:,.02f} km'.format(
        totFiles, totPoints, totDist / 1000))

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(spikes) > 0:
        print(os.linesep * 2 + 'Possible spikes:')
        for spike in spikes:
            print(spike + ',' + spikes[spike])

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    findSpikes()
    
    print(os.linesep + 'All done!')

........S.......S...S.....S.......................................................................................................................S..........

Summary: files = 157, points = 2,113,927, distance = 3,579.67 km


Possible spikes:
MOR854PIE_854_20240424_083052.oao,22.76,44.24,81.94,0.88,3.47,1.12,11
KOL827LUI_827_20240424_083022.oao,17.85,34.69,64.25,1.39,0.22,0.57,23
PRU801AIV_801_20240424_083036.oao,18.48,35.93,66.54,0.86,0.29,0.55,23
BOR810CHR_810_20240424_083114.oao,21.28,41.36,76.60,0.85,2.41,0.73,17
TOR836MEL_836_20240423_083942.oao,18.64,36.22,67.09,0.78,1.26,0.67,20

All done!
